#Imports and Uploads

In [ ]:
'''
The four files need to be uploaded onto the colab environment:
COG.csv, PHENO.csv, MRI.csv, DEMO.csv
It can be uploaded by running this cell
'''
from google.colab import files
m,p,d,i = files.upload()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from keras.models import Model
from keras.layers import Input
from keras.layers import GRU
from scipy.stats import pearsonr
from sklearn.impute import KNNImputer

In [ ]:
#Once the files are uploaded onto the environment
pheno = pd.read_csv('PHENO.csv')
mri = pd.read_csv('MRI.csv')
demo = pd.read_csv('DEMO.csv')
cog = pd.read_csv('COG.csv')

#Correlation:

In [ ]:
cog = pd.read_csv('COG.csv')
mri = pd.read_csv('MRI.csv')
pheno = pd.read_csv('PHENO.csv')
demo = pd.read_csv('DEMO.csv')

co = list(cog.columns)
mr = list(mri.columns)
ph = list(pheno.columns)
de = list(demo.columns)

myset = {'Unnamed: 0','RID','VISCODE','DX_bl','PTGENDER'}
co = [ele for ele in co if ele not in myset]
mr = [ele for ele in mr if ele not in myset]
ph = [ele for ele in ph if ele not in myset]
de = [ele for ele in de if ele not in myset]

print("Within modal correlation:\n")

coeff, _ = pearsonr(cog['EcogPtTotal'],cog['EcogPtMem'])
print("Between ",co[0]," and ",co[1]," = ",coeff)
coeff, _ = pearsonr(mri['Hippocampus'],mri['Entorhinal'])
print("Between ",mr[0]," and ",mr[1]," = ",coeff)
for n in range(len(ph)):
	for m in range(n+1,len(ph)):
		coeff, _ = pearsonr(pheno[ph[n]],pheno[ph[m]])
		print("Between ",ph[n]," and ",ph[m]," = ",coeff)
for n in range(len(de)):
	for m in range(n+1,len(de)):
		coeff, _ = pearsonr(demo[de[n]],demo[de[m]])
		print("Between ",de[n]," and ",de[m]," = ",coeff)

print("\nBetween modal correlation:\n")

mer = pd.merge(cog[['RID','EcogPtMem','EcogPtTotal']], mri[['RID','Hippocampus','Entorhinal']], how = 'inner')
for n in range(len(co)):
	for m in range(len(mr)):
		coeff, _ = pearsonr(mer[co[n]],mer[mr[m]])
		print("Between ",co[n]," and ",mr[m]," = ",coeff)

mer = pd.merge(cog[['RID','EcogPtMem','EcogPtTotal']], pheno[['RID','ABETA','TAU','PTAU']], how = 'inner')
for n in range(len(co)):
	for m in range(len(ph)):
		coeff, _ = pearsonr(mer[co[n]],mer[ph[m]])
		print("Between ",co[n]," and ",ph[m]," = ",coeff)

mer = pd.merge(cog[['RID','EcogPtMem','EcogPtTotal']], demo[['RID','AGE','PTEDUCAT','APOE4']], how = 'inner')
for n in range(len(co)):
	for m in range(len(de)):
		coeff, _ = pearsonr(mer[co[n]],mer[de[m]])
		print("Between ",co[n]," and ",de[m]," = ",coeff)

mer = pd.merge(mri[['RID','Hippocampus','Entorhinal']], pheno[['RID','ABETA','TAU','PTAU']])
for n in range(len(mr)):
	for m in range(len(ph)):
		coeff, _ = pearsonr(mer[mr[n]],mer[ph[m]])
		print("Between ",mr[n]," and ",ph[m]," = ",coeff)

mer = pd.merge(mri[['RID','Hippocampus','Entorhinal']], demo[['RID','AGE','PTEDUCAT','APOE4']])
for n in range(len(mr)):
	for m in range(len(de)):
		coeff, _ = pearsonr(mer[mr[n]],mer[de[m]])
		print("Between ",mr[n]," and ",de[m]," = ",coeff)

mer = pd.merge(pheno[['RID','ABETA','TAU','PTAU']], demo[['RID','AGE','PTEDUCAT','APOE4']])
for n in range(len(ph)):
	for m in range(len(de)):
		coeff, _ = pearsonr(mer[ph[n]],mer[de[m]])
		print("Between ",ph[n]," and ",de[m]," = ",coeff)

#Extracting Feature Vectors:

##Cog:

#Dataset

In [ ]:
cog = cog.sort_values(by = ['RID'])

###Graph

In [ ]:
sns.catplot(x='VISCODE', y='EcogPtMem', data=cog, hue="DX_bl", height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(x='VISCODE', y='EcogPtTotal', data=cog, hue="DX_bl", height=8.27, aspect=11.7/8.27)

###Data formatting

In [ ]:
'''
Formatting the data as well as removing the data which contain too many missing 
values.
'''
CogPtMem = []
CogPtTotal = []
labels = []
start = 2
rids_cog = []
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTMEM
l2 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTTOT
cnt = 0
for index,row in cog.iterrows():
  if start != row['RID']:    
    if sum(l1) != 0 and sum(l2) != 0 and cnt>2:
      rids_cog.append(row['RID'])
      CogPtMem.append(l1)
      CogPtTotal.append(l2)
      if dec == 'AD':
        labels.append(1)
      else:
        labels.append(0)
    l1 = [0,0,0,0,0,0]
    l2 = [0,0,0,0,0,0]
    start = row['RID']
    cnt=0
  dec = row['DX_bl']
  m = row['VISCODE']
  ptm = row['EcogPtMem']
  ptt = row['EcogPtTotal']
  if m == 'bl':
    cnt+=1
    l1[0] = ptm
    l2[0] = ptt
  elif m == 'm06':
    cnt+=1
    l1[1] = ptm
    l2[1] = ptt
  elif m == 'm12':
    cnt+=1
    l1[2] = ptm
    l2[2] = ptt
  elif m == 'm24':
    cnt+=1
    l1[3] = ptm
    l2[3] = ptt
  elif m == 'm36':
    cnt+=1
    l1[4] = ptm
    l2[4] = ptt
  elif m == 'm48':
    cnt+=1
    l1[5] = ptm
    l2[5] = ptt

  

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from collections import Counter
from numpy import where
from matplotlib import pyplot
# define dataset
X_smote = np.concatenate((np.array(CogPtMem), np.array(CogPtTotal)), axis=1)
y_smote = labels

print(X_smote)
print(y_smote)

# summarize class distribution
counter = Counter(y_smote)
print(counter)

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_smote == label)[0]
	pyplot.scatter(X_smote[row_ix, 0], X_smote[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

# transform the dataset
oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X_smote, y_smote)

print(X_smote)
print(y_smote)

# summarize the new class distribution
counter = Counter(y_smote)
print(counter)

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_smote == label)[0]
	pyplot.scatter(X_smote[row_ix, 0], X_smote[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

In [ ]:
newarr = np.array_split(X_smote, 2, axis=1)
x1 = newarr[0]
x2 = newarr[1]
#Length of data in Cog
print(len(x1),len(x2))
y_cog = y_smote

In [ ]:
#KNN Imputation is done to fill in the few missing values which are left
imputer = KNNImputer(n_neighbors=28, missing_values=0, weights = 'distance')
x1_cog = imputer.fit_transform(x1)  #CogPtMem
x2_cog = imputer.fit_transform(x2)  #CogPtTotal

###Model_cog

In [ ]:
print("Shape of input:",len(x1_cog),len(x1_cog[0]))

In [ ]:
#GRUs require 3 dimensional input
x1_cog = x1_cog.reshape((1,1430,6))
x2_cog = x2_cog.reshape((1,1430,6))

In [ ]:
#Generating feature vectors
inputs1 = Input(shape=(1430, 6))
gru1 = GRU(6, return_sequences=True, return_state=True)
whole_sequence_output, final_state = gru1(inputs1)
model1 = Model(inputs=inputs1, outputs=whole_sequence_output)

In [ ]:
cogptmem_fv = model1.predict(x1_cog) #CogPtMem: fv = feature vector
cogpttotal_fv = model1.predict(x2_cog) #CogPtTotal

In [ ]:
from sklearn.model_selection import train_test_split
# 70% training and 30% test
CogPtMem_1 = cogptmem_fv[0]
cogpttotal_1 = cogpttotal_fv[0]
cog_fv = np.concatenate((CogPtMem_1, cogpttotal_1), axis=1)
print(y_cog)
print(cog_fv.shape)
X_train, X_test, y_train, y_test = train_test_split(cog_fv, y_cog, test_size=0.3,random_state=109)

#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train, y_train)

In [ ]:
# Actual class predictions
rf_predictions = model.predict(X_test)
# Probabilities for each class
rf_probs = model.predict_proba(X_test)[:, 1]

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import f1_score, cohen_kappa_score
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, permutation_test_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, rf_predictions))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, rf_predictions))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, rf_predictions))

#Printing Confusion Matrix
conf_matrix_rf = metrics.confusion_matrix(y_test, rf_predictions)
print("Confusion Matrix: ", conf_matrix_rf )

#Printing F1 Score
f1score = f1_score(y_test, rf_predictions, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, rf_predictions)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, rf_predictions)
print("Average Precision: ", average_precision)
disp = plot_precision_recall_curve(model, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(model, X_test, y_test)
plt.show()

#P-value
_, _, p_value = permutation_test_score(model, X_test, y_test, scoring="neg_root_mean_squared_error", cv=None, n_permutations=200)

print(p_value)

#SVM

In [ ]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred1 = clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred1))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred1))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred1))

#Printing Confusion Matrix
conf_matrix_svm = metrics.confusion_matrix(y_test, y_pred1)
print("Confusion Matrix: ", conf_matrix_svm )

#Printing F1 Score
f1score = f1_score(y_test, y_pred1, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, y_pred1)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, y_pred1)
disp = plot_precision_recall_curve(clf, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(clf, X_test, y_test)
plt.show()


#P-value
_, _, p_value = permutation_test_score(clf, X_test, y_test, scoring="neg_root_mean_squared_error", cv=None, n_permutations=100)
print(p_value)

#Decision Tree

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz 
from IPython.display import Image 
from pydot import graph_from_dot_data
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
y_pred = dt.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

#Printing Confusion Matrix
conf_matrix_dt = metrics.confusion_matrix(y_test, y_pred)
print("Confusion Matrix: ", conf_matrix_dt )

#Printing F1 Score
f1score = f1_score(y_test, y_pred, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, y_pred)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, y_pred)
disp = plot_precision_recall_curve(dt, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(dt, X_test, y_test)
plt.show()

#P-value
_, _, p_value = permutation_test_score(dt, X_test, y_test, scoring="neg_root_mean_squared_error", cv=None, n_permutations=200)
print(p_value)

##Ensemble Model

In [ ]:
from sklearn.linear_model import LogisticRegression
import six
import sys
sys.modules['sklearn.externals.six'] = six
from mlxtend.classifier import StackingClassifier
lr = LogisticRegression()
clf_stack = StackingClassifier(classifiers =[clf, dt, model], meta_classifier = lr, use_probas = False, use_features_in_secondary = True)
model_stack = clf_stack.fit(X_train, y_train)
pred_stack = model_stack.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, pred_stack))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, pred_stack))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, pred_stack))

#Printing Confusion Matrix
conf_matrix_dt = metrics.confusion_matrix(y_test, pred_stack)
print("Confusion Matrix: ", conf_matrix_dt )

#Printing F1 Score
f1score = f1_score(y_test, pred_stack, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, pred_stack)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, pred_stack)
disp = plot_precision_recall_curve(model_stack, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(model_stack, X_test, y_test)
plt.show()

#Making Combined Graphs of models.

In [ ]:
#ROC Curve

from sklearn import metrics
import matplotlib.pyplot as plt

plt.figure()


from sklearn.metrics import plot_roc_curve
classifiers = [clf, dt, model]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X_test, y_test, ax=ax)

In [ ]:
#Precision Recall Curve

from sklearn import metrics
import matplotlib.pyplot as plt

plt.figure()


from sklearn.metrics import plot_precision_recall_curve
classifiers = [clf, dt, model]
ax = plt.gca()
for i in classifiers:
    plot_precision_recall_curve(i, X_test, y_test, ax=ax)

##Pheno:

In [ ]:
#dataset
pheno = pheno.sort_values(by = ['RID'])

In [ ]:
sns.catplot(x='VISCODE', y='ABETA', data=pheno, hue="DX_bl", height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(x='VISCODE', y='TAU', data=pheno, hue="DX_bl", height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(x='VISCODE', y='PTAU', data=pheno, hue="DX_bl", height=8.27, aspect=11.7/8.27)

###Data formatting

In [ ]:
'''
Formatting the data as well as removing the data which contain too many missing 
values.
'''
abeta = []
tau = []
ptau = []
labels = []
start = 3
rids_pheno = []
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 abeta
l2 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 tau
l3 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 ptau
cnt = 0
for index,row in pheno.iterrows():
  if start != row['RID']:    
    if sum(l1) != 0 and sum(l2) != 0 and sum(l3) != 0 and cnt>2:
      rids_pheno.append(row['RID'])
      abeta.append(l1)
      tau.append(l2)
      ptau.append(l3)
      if dec == 'AD':
        labels.append(1)
      else:
        labels.append(0)
    l1 = [0,0,0,0,0,0]
    l2 = [0,0,0,0,0,0]
    l3 = [0,0,0,0,0,0]
    start = row['RID']
    cnt=0
  dec = row['DX_bl']
  m = row['VISCODE']
  abt = row['ABETA']
  ta = row['TAU']
  pta = row['PTAU']
  if m == 'bl':
    cnt+=1
    l1[0] = abt
    l2[0] = ta
    l3[0] = pta
  elif m == 'm06':
    cnt+=1
    l1[1] = abt
    l2[1] = ta
    l3[1] = pta
  elif m == 'm12':
    cnt+=1
    l1[2] = abt
    l2[2] = ta
    l3[2] = pta
  elif m == 'm24':
    cnt+=1
    l1[3] = abt
    l2[3] = ta
    l3[3] = pta
  elif m == 'm36':
    cnt+=1
    l1[4] = abt
    l2[4] = ta
    l3[4] = pta
  elif m == 'm48':
    cnt+=1
    l1[5] = abt
    l2[5] = ta
    l3[5] = pta

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from collections import Counter
from numpy import where
from matplotlib import pyplot
# define dataset
X_smote = np.concatenate((np.array(abeta), np.array(tau), np.array(ptau)), axis=1)
y_smote = labels

print(X_smote)
print(y_smote)

# summarize class distribution
counter = Counter(y_smote)
print(counter)

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_smote == label)[0]
	pyplot.scatter(X_smote[row_ix, 0], X_smote[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

# transform the dataset
oversample = SMOTE(sampling_strategy=0.5)
X_smote, y_smote = oversample.fit_resample(X_smote, y_smote)

print(X_smote)
print(y_smote)

# summarize the new class distribution
counter = Counter(y_smote)
print(counter)

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_smote == label)[0]
	pyplot.scatter(X_smote[row_ix, 0], X_smote[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

In [ ]:
newarr = np.array_split(X_smote, 3, axis=1)
x1 = newarr[0]
x2 = newarr[1]
x3 = newarr[2]
#Length of data in Pheno
y_pheno = y_smote

In [ ]:
#Imputation
imputer = KNNImputer(n_neighbors=16, missing_values=0, weights = 'distance')
x1_pheno = imputer.fit_transform(x1) #ABETA
x2_pheno = imputer.fit_transform(x2) #TAU
x3_pheno = imputer.fit_transform(x3) #PTAU

###Model_pheno

In [ ]:
print("Shape of input:",len(x1_pheno),len(x1_pheno[0]))

In [ ]:
#Reshaping for GRU
x1_pheno = x1_pheno.reshape((1,337,6))
x2_pheno = x2_pheno.reshape((1,337,6))
x3_pheno = x3_pheno.reshape((1,337,6))

In [ ]:
#Normalizing all data
x1_pheno = x1_pheno/400
x2_pheno = x2_pheno/200
x3_pheno = x3_pheno/50

In [ ]:
#Generating feature vectors
inputs2 = Input(shape=(337, 6))
gru2 = GRU(6, return_sequences=True, return_state=True)
whole_sequence_output, final_state = gru2(inputs2)
model2 = Model(inputs=inputs2, outputs=whole_sequence_output)

In [ ]:
abeta_fv = model2.predict(x1_pheno)
tau_fv = model2.predict(x2_pheno)
ptau_fv = model2.predict(x3_pheno)

#Random Forest Classifier

In [ ]:
from sklearn.model_selection import train_test_split
# 70% training and 30% test
y_pheno = y_smote
abeta_fv1 = abeta_fv[0]
tau_fv1 = tau_fv[0]
ptau_fv1 = ptau_fv[0]
pheno_fv = np.concatenate((abeta_fv1, tau_fv1, ptau_fv1), axis=1)
X_train, X_test, y_train, y_test = train_test_split(pheno_fv, y_pheno, test_size=0.3,random_state=109)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train, y_train)

In [ ]:
# Actual class predictions
rf_predictions = model.predict(X_test)
# Probabilities for each class
rf_probs = model.predict_proba(X_test)[:, 1]

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.model_selection import train_test_split, permutation_test_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, rf_predictions))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, rf_predictions))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, rf_predictions))

#Printing Confusion Matrix
conf_matrix = metrics.confusion_matrix(y_test, rf_predictions)
print("Confusion Matrix: ", conf_matrix )

#Printing F1 Score
f1score = f1_score(y_test, rf_predictions, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, rf_predictions)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, rf_predictions)
disp = plot_precision_recall_curve(model, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(model, X_test, y_test)
plt.show()


#P-value
_, _, p_value = permutation_test_score(model, X_test, y_test, scoring="neg_root_mean_squared_error", cv=None, n_permutations=200)
print(p_value)

###SVM Classifier

In [ ]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

#Printing Confusion Matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Confusion Matrix: ", conf_matrix )

#Printing F1 Score
f1score = f1_score(y_test, y_pred, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, y_pred)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, y_pred)
disp = plot_precision_recall_curve(clf, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(clf, X_test, y_test)
plt.show()


#P-value
_, _, p_value = permutation_test_score(clf, X_test, y_test, scoring="neg_root_mean_squared_error", cv=None, n_permutations=200)
print(p_value)

#Decision Tree

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from IPython.display import Image 
from pydot import graph_from_dot_data
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
y_pred_dt = dt.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_dt))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred_dt))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred_dt))

#Printing Confusion Matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred_dt)
print("Confusion Matrix: ", conf_matrix )

#Printing F1 Score
f1score = f1_score(y_test, y_pred_dt, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, y_pred_dt)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, y_pred_dt)
disp = plot_precision_recall_curve(dt, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(dt, X_test, y_test)
plt.show()

#P-value
_, _, p_value = permutation_test_score(dt, X_test, y_test, scoring="neg_root_mean_squared_error", cv=None, n_permutations=200)
print(p_value)

##Ensemble Model

In [ ]:
from sklearn.linear_model import LogisticRegression
import six
import sys
sys.modules['sklearn.externals.six'] = six
from mlxtend.classifier import StackingClassifier
lr = LogisticRegression()
clf_stack = StackingClassifier(classifiers =[clf, dt, model], meta_classifier = lr, use_probas = False, use_features_in_secondary = True)
model_stack = clf_stack.fit(X_train, y_train)
pred_stack = model_stack.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, pred_stack))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, pred_stack))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, pred_stack))

#Printing Confusion Matrix
conf_matrix_dt = metrics.confusion_matrix(y_test, pred_stack)
print("Confusion Matrix: ", conf_matrix_dt )

#Printing F1 Score
f1score = f1_score(y_test, pred_stack, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, pred_stack)
print("Cohen-Kappa Score: ", ckscore)

#Forming Combined Graphs

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
plt.figure()

from sklearn.metrics import plot_roc_curve
classifiers = [clf, dt, model]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X_test, y_test, ax=ax)

In [ ]:
#Precision Recall Curve

from sklearn import metrics
import matplotlib.pyplot as plt

plt.figure()


from sklearn.metrics import plot_precision_recall_curve
classifiers = [clf, dt, model]
ax = plt.gca()
for i in classifiers:
    plot_precision_recall_curve(i, X_test, y_test, ax=ax)

##MRI:

In [ ]:
mri = mri.sort_values(by=['RID'])

###Graph

In [ ]:
sns.catplot(x='VISCODE', y='Hippocampus', data=mri, hue="DX_bl", height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(x='VISCODE', y='Entorhinal', data=mri, hue="DX_bl", height=8.27, aspect=11.7/8.27)

###Data formatting

In [ ]:
hippo = []
entor = []
labels = []
start = 2
rids_mri = []
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTMEM
l2 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTTOT
cnt = 0
for index,row in mri.iterrows():
  if start != row['RID']:    
    if sum(l1) != 0 and sum(l2) != 0 and cnt>2:
      rids_mri.append(row['RID'])
      hippo.append(l1)
      entor.append(l2)
      if dec == 'AD':
        labels.append(1)
      else:
        labels.append(0)
    l1 = [0,0,0,0,0,0]
    l2 = [0,0,0,0,0,0]
    start = row['RID']
    cnt=0
  dec = row['DX_bl']
  m = row['VISCODE']
  hi = row['Hippocampus']
  en = row['Entorhinal']
  if m == 'bl':
    cnt+=1
    l1[0] = hi
    l2[0] = en
  elif m == 'm06':
    cnt+=1
    l1[1] = hi
    l2[1] = en
  elif m == 'm12':
    cnt+=1
    l1[2] = hi
    l2[2] = en
  elif m == 'm24':
    cnt+=1
    l1[3] = hi
    l2[3] = en
  elif m == 'm36':
    cnt+=1
    l1[4] = hi
    l2[4] = en
  elif m == 'm48':
    cnt+=1
    l1[5] = hi
    l2[5] = en

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from collections import Counter
from numpy import where
from matplotlib import pyplot
# define dataset
X_smote = np.concatenate((np.array(hippo), np.array(entor)), axis=1)
y_smote = labels

print(X_smote)
print(y_smote)

# summarize class distribution
counter = Counter(y_smote)
print(counter)

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_smote == label)[0]
	pyplot.scatter(X_smote[row_ix, 0], X_smote[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

# transform the dataset
oversample = SMOTE(sampling_strategy=0.5)
X_smote, y_smote = oversample.fit_resample(X_smote, y_smote)

print(X_smote)
print(y_smote)

# summarize the new class distribution
counter = Counter(y_smote)
print(counter)

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_smote == label)[0]
	pyplot.scatter(X_smote[row_ix, 0], X_smote[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

In [ ]:
newarr = np.array_split(X_smote, 2, axis=1)
x1_mri = newarr[0]
x2_mri = newarr[1]
#Length of data in Cog
print(x1_mri)
print(len(x1_mri),len(x2_mri))
y_mri = y_smote

In [ ]:
imputer = KNNImputer(n_neighbors=32, missing_values=0, weights = 'distance')
x1_mri = imputer.fit_transform(x1_mri) #Hippocampus
x2_mri = imputer.fit_transform(x2_mri) #Entorhinal

###Model_mri

In [ ]:
print("Shape of input:",len(x1_mri),len(x1_mri[0]))

In [ ]:
#Reshaping for GRU
x1_mri = x1_mri.reshape((1,1357,6))
x2_mri = x2_mri.reshape((1,1357,6))

In [ ]:
#Normalizing all data
x1_mri = x1_mri/2000
x2_mri = x2_mri/2000

In [ ]:
#Generating feature vectors
inputs3 = Input(shape=(1357, 6))
gru3 = GRU(6, return_sequences=True, return_state=True)
whole_sequence_output, final_state = gru3(inputs3)
model3 = Model(inputs=inputs3, outputs=whole_sequence_output)

In [ ]:
hippo_fv = model3.predict(x1_mri)
entor_fv = model3.predict(x2_mri)

#Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
# 70% training and 30% test
hippo_1 = hippo_fv[0]
entor_1 = entor_fv[0]
mri_fv = np.concatenate((hippo_1, entor_1), axis=1)
X_train, X_test, y_train, y_test = train_test_split(mri_fv, y_mri, test_size=0.3,random_state=109)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train, y_train)

In [ ]:
# Actual class predictions
rf_predictions = model.predict(X_test)
# Probabilities for each class
rf_probs = model.predict_proba(X_test)[:, 1]

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, rf_predictions))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, rf_predictions))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, rf_predictions))

#Printing Confusion Matrix
conf_matrix = metrics.confusion_matrix(y_test, rf_predictions)
print("Confusion Matrix: ", conf_matrix )

#Printing F1 Score
f1score = f1_score(y_test, rf_predictions, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, rf_predictions)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, rf_predictions)
disp = plot_precision_recall_curve(model, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(model, X_test, y_test)
plt.show()

#P-value
_, _, p_value = permutation_test_score(model, X_test, y_test, scoring="neg_root_mean_squared_error", cv=None, n_permutations=100)
print(p_value)

#SVM Model

In [ ]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

#Printing Confusion Matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Confusion Matrix: ", conf_matrix )

#Printing F1 Score
f1score = f1_score(y_test, y_pred, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, y_pred)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, y_pred)
disp = plot_precision_recall_curve(clf, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(clf, X_test, y_test)
plt.show()

#P-value
_, _, p_value = permutation_test_score(clf, X_test, y_test, scoring="neg_root_mean_squared_error", cv=None, n_permutations=200)
print(p_value)

#Decision Tree

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from IPython.display import Image 
from pydot import graph_from_dot_data
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
y_pred_dt = dt.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_dt))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred_dt))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred_dt))

#Printing Confusion Matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred_dt)
print("Confusion Matrix: ", conf_matrix )

#Printing F1 Score
f1score = f1_score(y_test, y_pred_dt, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, y_pred_dt)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, y_pred_dt)
disp = plot_precision_recall_curve(dt, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(dt, X_test, y_test)
plt.show()

#P-value
_, _, p_value = permutation_test_score(dt, X_test, y_test, scoring="neg_root_mean_squared_error", cv=None, n_permutations=200)
print(p_value)

##Ensemble Model

In [ ]:
from sklearn.linear_model import LogisticRegression
import six
import sys
sys.modules['sklearn.externals.six'] = six
from mlxtend.classifier import StackingClassifier
lr = LogisticRegression()
clf_stack = StackingClassifier(classifiers =[clf, dt, model], meta_classifier = lr, use_probas = False, use_features_in_secondary = True)
model_stack = clf_stack.fit(X_train, y_train)
pred_stack = model_stack.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, pred_stack))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, pred_stack))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, pred_stack))

#Printing Confusion Matrix
conf_matrix_dt = metrics.confusion_matrix(y_test, pred_stack)
print("Confusion Matrix: ", conf_matrix_dt )

#Printing F1 Score
f1score = f1_score(y_test, pred_stack, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, pred_stack)
print("Cohen-Kappa Score: ", ckscore)

#Forming Combined Graphs

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
plt.figure()

from sklearn.metrics import plot_roc_curve
classifiers = [clf, dt, model]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X_test, y_test, ax=ax)

In [ ]:
#Precision Recall Curve

from sklearn import metrics
import matplotlib.pyplot as plt

plt.figure()


from sklearn.metrics import plot_precision_recall_curve
classifiers = [clf, dt, model]
ax = plt.gca()
for i in classifiers:
    plot_precision_recall_curve(i, X_test, y_test, ax=ax)